In [1]:
!pip install kaggle


In [3]:
#configuring the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
#API to fetch thedataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 96% 78.0M/80.9M [00:00<00:00, 194MB/s]
100% 80.9M/80.9M [00:00<00:00, 190MB/s]


In [5]:
#extracting the compressed dataset
from zipfile import ZipFile
dataset='/content/sentiment140.zip'
with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [8]:
#importing the dependencies
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression



In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
#printing the stopwords in english
print(stopwords.words('english'))


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [14]:
#Data processsing
#loading the data from csv file to pandas dataframe
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')


In [16]:
#checkin the number of rows and columns
twitter_data.shape

(1599999, 6)

In [17]:
#printing the first 5 rows of the dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [19]:
#naming the colunms and reading the dataset again
column_names=['tagert', 'id', 'date', 'flag', 'user', 'text']
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names=column_names,encoding='ISO-8859-1')

In [20]:
twitter_data.shape

(1600000, 6)

In [21]:
twitter_data.head()

,tagert,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [22]:
#counting the number of missing values in the dataset
twitter_data.isnull().sum()

,0
tagert,0
id,0
date,0
flag,0
user,0
text,0


In [27]:
#checking the distribution of target column
twitter_data['tagert'].value_counts()

,count
tagert,
0,800000
4,800000


In [29]:
#convert the target "4" to "1"
twitter_data.replace({'tagert':{4:1}},inplace=True)

In [30]:
#checking the distribution of target column
twitter_data['tagert'].value_counts()

,count
tagert,
0,800000
1,800000


In [31]:
#0 -->Negative Tweet
#1 -->Positive Tweet

#Stemming : process of reducing a word to its Root word
port_stem = PorterStemmer()


In [34]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]', ' ', content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word)for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [35]:
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming)

In [37]:
twitter_data.head()

,tagert,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [38]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [39]:
print(twitter_data['tagert'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: tagert, Length: 1600000, dtype: int64


In [40]:
#separating the data and label
X =twitter_data['stemmed_content'].values

Y =twitter_data['tagert'].values

In [42]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [41]:
print(Y)

[0 0 0 ... 1 1 1]


In [45]:
#splitting the data to training data and test data

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)


In [46]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [47]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [53]:
#converting the textual data to numerical data
from sklearn.feature_extraction.text import TfidfVectorizer

# Assurez-vous que X_train et X_test sont des listes de texte brut à ce stade
vectorizer = TfidfVectorizer()

# Appliquer fit_transform sur X_train (texte brut)
X_train = vectorizer.fit_transform(X_train)

# Transformer X_test en utilisant le même vectorizer (texte brut)
X_test = vectorizer.transform(X_test)



AttributeError: 'csr_matrix' object has no attribute 'lower'

In [50]:
print(X_train)

  (0, 436713)	0.27259876264838384
  (0, 354543)	0.3588091611460021
  (0, 185193)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (0, 235045)	0.41996827700291095
  (0, 443066)	0.4484755317023172
  (1, 160636)	1.0
  (2, 109306)	0.4591176413728317
  (2, 124484)	0.1892155960801415
  (2, 407301)	0.18709338684973031
  (2, 129411)	0.29074192727957143
  (2, 406399)	0.32105459490875526
  (2, 433560)	0.3296595898028565
  (2, 77929)	0.31284080750346344
  (2, 443430)	0.3348599670252845
  (2, 266729)	0.24123230668976975
  (2, 409143)	0.15169282335109835
  (2, 178061)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.2028971570399794
  (2, 288470)	0.16786949597862733
  (3, 406399)	0.29029991238662284
  (3, 158711)	0.4456939372299574
  (3, 151770)	0.278559647704793
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 318303)	0.21254698865277744
  (1279996, 434014)	0.27189450523324465
  (1279996, 390130)	0.2206474219107611
  (1279996, 373144)	0.35212500999832036
  (1279996, 23807

In [54]:
print(X_test)

  (0, 15110)	0.1719352837797837
  (0, 31168)	0.1624772418052177
  (0, 67828)	0.26800375270827315
  (0, 106069)	0.36555450010904555
  (0, 132364)	0.255254889555786
  (0, 138164)	0.23688292264071406
  (0, 171378)	0.2805816206356074
  (0, 271016)	0.45356623916588285
  (0, 279082)	0.17825180109103442
  (0, 388348)	0.2198507607206174
  (0, 398906)	0.34910438732642673
  (0, 409143)	0.3143047059807971
  (0, 420984)	0.17915624523539805
  (1, 6463)	0.30733520460524466
  (1, 15110)	0.211037449588008
  (1, 145393)	0.575262969264869
  (1, 217562)	0.40288153995289894
  (1, 256777)	0.28751585696559306
  (1, 348135)	0.4739279595416274
  (1, 366203)	0.24595562404108307
  (2, 22532)	0.3532582957477176
  (2, 34401)	0.37916255084357414
  (2, 89448)	0.36340369428387626
  (2, 183312)	0.5892069252021465
  (2, 256834)	0.2564939661498776
  :	:
  (319994, 443794)	0.2782185641032538
  (319995, 107868)	0.33399349737546963
  (319995, 109379)	0.3020896484890833
  (319995, 155493)	0.2770682832971669
  (319995, 2133

In [55]:
#trainig the machine learning modele
#logistic regression
model=LogisticRegression(max_iter=1000)


In [56]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

In [59]:
#model evaluation
# accuracy score on the trainig data
# Importer accuracy_score depuis sklearn.metrics
from sklearn.metrics import accuracy_score

# Évaluation du modèle
# Prédiction sur les données d'entraînement
X_train_prediction = model.predict(X_train)

# Calcul de l'accuracy sur les données d'entraînement
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)



In [60]:
print('Accuracy score on the training data : ', training_data_accuracy)

Accuracy score on the training data :  0.79871953125


In [61]:
#model evaluation
# accuracy score on the testing data
X_test_prediction=model.predict(X_test)
test_data_accuracy= accuracy_score( Y_test, X_test_prediction)

In [62]:
print('Accuracy score on the training data : ', test_data_accuracy)

Accuracy score on the training data :  0.77668125


In [64]:
#model accuracy = 77.8%

#saving the trained model
# Importer le module pickle
import pickle

# Sauvegarder le modèle entraîné
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

print("Model saved successfully!")


Model saved successfully!


In [65]:
#using the saved model for future predictions
#loading the saed model
loaded_model=pickle.load(open('/content/trained_model.sav', 'rb'))

In [66]:
X_new = X_test[200]
print(Y_test[200])
prediction = model.predict(X_new)
print(prediction)
if(prediction[0]==0):
  print('negative tweet')
  print('positive tweet')

1
[1]


In [67]:
!git clone https://github.com/sana-rekbi/Twitter-Sentiment-Analysis-.git


Cloning into 'Twitter-Sentiment-Analysis-'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 13 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (13/13), 6.14 KiB | 6.14 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [69]:
%cd Twitter-Sentiment-Analysis-


[Errno 2] No such file or directory: 'Twitter-Sentiment-Analysis-'
/content/Twitter-Sentiment-Analysis-


In [70]:
!git checkout -b model-branch


Switched to a new branch 'model-branch'


In [73]:
from google.colab import files
files.download('Twitter_sentiment_analysis.ipynb')


FileNotFoundError: Cannot find file: Twitter_sentiment_analysis.ipynb

In [ ]:
!mv twitter_sentiments_analysis.ipynb Twitter-Sentiment-Analysis-
